In [1]:
                               # IMPORTING LIBRARIES

In [2]:
import numpy as np
import os
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Model
from tensorflow.keras.utils import img_to_array, load_img
from shutil import copyfile
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

%matplotlib inline

In [3]:
#Defining variables

In [4]:
BATCH_SIZE = 32 
EPOCHS = 3
IMAGE_SIZE = 256 
default_image_size = tuple((IMAGE_SIZE, IMAGE_SIZE)) 
image_size = 0 
CHANNELS=3 
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
# Set random seed for reproducibility

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
                              # LOADING DATASET

In [8]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE
)

Found 19553 files belonging to 4 classes.


In [9]:
                           #  LISTING THE CLASSES

In [10]:
class_names = dataset.class_names
class_names

['Cerscospora', 'Healthy', 'Leaf rust', 'Phoma']

In [11]:
                          # Data Preprocessing

In [12]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = ds.cardinality().numpy()
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [13]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "PlantVillage",
  seed=123,
  image_size=default_image_size,
  batch_size=BATCH_SIZE
)


train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)



Found 19553 files belonging to 4 classes.


In [14]:
# Training Image preprocessing

In [15]:
train_size=0.8
len(dataset)*train_size
train_ds=dataset.take(131)
len(train_ds)

131

In [16]:
# Testing Image preprocessing

In [17]:
test_ds=dataset.skip(16)
len(test_ds)

596

In [18]:
# Validation Image preprocessing


In [19]:
val_ds=test_ds.take(16)
len(val_ds)
test_ds=test_ds.skip(16)
len(test_ds)

580

In [20]:
                                         # Building Model

In [21]:
                                    # Building Convolution Layer

In [22]:
                          # Compiling and Training Phase

In [23]:
class_names = dataset.class_names
n_classes = len(class_names)
print(n_classes, class_names)

4 ['Cerscospora', 'Healthy', 'Leaf rust', 'Phoma']


In [24]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [25]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [26]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])


In [27]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
batch_input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
chanDim = -1
if K.image_data_format() == "channels_first":
    input_shape = (CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    batch_input_shape = (BATCH_SIZE, CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    chanDim = 1

In [28]:
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=batch_input_shape)

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (32, 127, 127, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (32, 62, 62, 64)          0         
 g2D)                                                 

In [30]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [31]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=15,  
)

Epoch 1/15
131/131 [==============================] - 604s 4s/step - loss: 0.6122 - accuracy: 0.7696 - val_loss: 0.5177 - val_accuracy: 0.8008
Epoch 2/15
131/131 [==============================] - 589s 4s/step - loss: 0.3375 - accuracy: 0.8915 - val_loss: 0.2986 - val_accuracy: 0.8809
Epoch 3/15
131/131 [==============================] - 382s 3s/step - loss: 0.2612 - accuracy: 0.9105 - val_loss: 0.2803 - val_accuracy: 0.9316
Epoch 4/15
131/131 [==============================] - 385s 3s/step - loss: 0.2034 - accuracy: 0.9358 - val_loss: 0.2064 - val_accuracy: 0.9395
Epoch 5/15
131/131 [==============================] - 387s 3s/step - loss: 0.2055 - accuracy: 0.9356 - val_loss: 0.1890 - val_accuracy: 0.9453
Epoch 6/15
131/131 [==============================] - 363s 3s/step - loss: 0.1907 - accuracy: 0.9389 - val_loss: 0.2586 - val_accuracy: 0.9395
Epoch 7/15
131/131 [==============================] - 446s 3s/step - loss: 0.1506 - accuracy: 0.9556 - val_loss: 0.1204 - val_accuracy: 0.9629

In [32]:
                                                # Saving Model 

In [33]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [34]:
#Training set Accuracy
scores = model.evaluate(train_ds)

131/131 [==============================] - 160s 1s/step - loss: 0.0238 - accuracy: 0.9914


In [35]:
#Validation set Accuracy
scores = model.evaluate(val_ds)

16/16 [==============================] - 28s 1s/step - loss: 0.0260 - accuracy: 0.9922


In [36]:
                                                 #Saving Model

In [37]:
import os
model_version = max([int(i) for i in (os.listdir(f"./Model")+[0])]) + 1
model.save(f"./Model/{model_version}")

INFO:tensorflow:Assets written to: ./saved_models/16\assets


INFO:tensorflow:Assets written to: ./saved_models/16\assets


In [38]:
                                                 #Saving H5 Model

In [39]:
model.save('coffee_model.h5')

C:\Users\820 G3\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [40]:
                                                 #Saving Keras

In [41]:
model.save('disease.keras')

In [42]:
                           # Converting Model to TFLite

In [43]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\820G3~1\AppData\Local\Temp\tmpql2xtcmv\assets


INFO:tensorflow:Assets written to: C:\Users\820G3~1\AppData\Local\Temp\tmpql2xtcmv\assets


In [44]:
with open("coffee.tflite",'wb') as f:
    f.write(tflite_model)

In [45]:
history.history #Return Dictionary of history

{'loss': [0.6122015118598938,
  0.3374897539615631,
  0.261168897151947,
  0.2033757120370865,
  0.20547881722450256,
  0.19065330922603607,
  0.1505882889032364,
  0.10420755296945572,
  0.0902390405535698,
  0.08543858677148819,
  0.06547963619232178,
  0.08911720663309097,
  0.06566452980041504,
  0.07085878401994705,
  0.05939154699444771],
 'accuracy': [0.7695610523223877,
  0.8914599418640137,
  0.9105439186096191,
  0.9358301758766174,
  0.9355915784835815,
  0.9389312863349915,
  0.9556297659873962,
  0.9718511700630188,
  0.9773377776145935,
  0.9763835668563843,
  0.9833015203475952,
  0.9744752049446106,
  0.9809160232543945,
  0.9794847369194031,
  0.9852099418640137],
 'val_loss': [0.5177059769630432,
  0.29857128858566284,
  0.280348002910614,
  0.20640863478183746,
  0.18903027474880219,
  0.25857609510421753,
  0.12040208280086517,
  0.07058554887771606,
  0.05768671631813049,
  0.05103199556469917,
  0.1825566291809082,
  0.1205848976969719,
  0.11156624555587769,
  0.

In [46]:
#Recording History in json
import json
with open('training_hist.json','w') as f:
  json.dump(history.history,f)
     

In [47]:
                          #Calculating Accuracy of Model Achieved on Validation set

In [48]:
print("Validation set Accuracy: {} %".format(history.history['val_accuracy'][-1]*100))

Validation set Accuracy: 99.21875 %


In [49]:
                    #Test set Evaluation

In [50]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(64, 64),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)
     

Found 19553 files belonging to 4 classes.


In [51]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [52]:
plt.figure(figsize=(10, 10))
for images, labels in test_ds.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i].numpy())
        
        actual_class = class_names[labels[i]] 
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        plt.axis("off")

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} jpeg::Uncompress failed. Invalid JPEG data or crop window.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 

<Figure size 1000x1000 with 0 Axes>

In [ ]:
                                               #Accuracy Visualization 

                                               #Training Visualization

In [ ]:
epochs = [i for i in range(0,15)]
plt.plot(epochs,history.history['accuracy'],color='red')
plt.xlabel('No. of Epochs')
plt.ylabel('Training Accuracy')
plt.title('Visualization of Training Accuracy Result')
plt.show()  

In [ ]:
                                                 #Validation Accuracy 

In [ ]:
plt.plot(epochs, history.history['val_accuracy'],color='blue')
plt.xlabel('No. of Epochs')
plt.ylabel('Validation Accuracy')
plt.title('Visualization of Validation Accuracy Result')
plt.show()

In [ ]:
                                                 #Training and Validation Accuracy and Loss

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
epochs = [i for i in range(0,15)]
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()